In [1]:
import os
import cv2
import shutil
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import pandas as pd
import glob
from PIL import Image
import matplotlib

In [2]:
# 1. Видалення порожніх .txt файлів з мітками
def remove_empty_label_files(folder_path):
    count = 0
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                full_path = os.path.join(root, file)
                if os.stat(full_path).st_size == 0:
                    os.remove(full_path)
                    count += 1
    print(f"[INFO] Removed {count} empty label files from {folder_path}")

remove_empty_label_files('./data/train/labels')
remove_empty_label_files('./data/val/labels')

[INFO] Removed 0 empty label files from ./data/train/labels
[INFO] Removed 0 empty label files from ./data/val/labels


In [3]:
# 2. Видалення зіпсованих або надто малих зображень
def remove_bad_images(image_dir):
    removed = 0
    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        try:
            img = cv2.imread(img_path)
            if img is None:
                print(f"[BAD] Can't read: {img_path} — deleting")
                os.remove(img_path)
                removed += 1
            else:
                h, w = img.shape[:2]
                if h < 10 or w < 10:
                    print(f"[BAD] Too small ({w}x{h}): {img_path} — deleting")
                    os.remove(img_path)
                    removed += 1
        except Exception as e:
            print(f"[ERROR] {img_path}: {e}")
    print(f"[INFO] Removed {removed} bad images from {image_dir}")

remove_bad_images('data/train/images')
remove_bad_images('data/val/images')

[INFO] Removed 0 bad images from data/train/images
[INFO] Removed 0 bad images from data/val/images


In [4]:
# 3. Основна логіка
def main():
    # 3.1 Створення YAML конфігурації
    data_yaml = {
        'train': './data/train',
        'val': './data/val',
        'nc': 10,
        'names': [
            'door', 'open_door', 'cabinet_door', 'fridge_door',
            'window', 'chair', 'table', 'cabinet', 'sofa', 'pillar'
        ]
    }

    if not os.path.exists('indoor.yaml'):
        with open('indoor.yaml', 'w') as f:
            yaml.dump(data_yaml, f)
        print("[INFO] Created indoor.yaml")
    else:
        print("[INFO] indoor.yaml already exists — skipping write.")

In [5]:
# 3.2 Завантаження моделі
print("[INFO] Loading YOLOv9s model...")
model = YOLO('yolov9s.pt')

[INFO] Loading YOLOv9s model...


In [6]:
# 3.3 Тренування
print("[INFO] Starting training...")
results = model.train(
    data='indoor.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='indoor_yolov9s6',
    device=0,      # GPU (або 'cpu')
    workers=0      # важливо для Windows!
)

[INFO] Starting training...
New https://pypi.org/project/ultralytics/8.3.169 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.167  Python-3.12.6 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=indoor.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=indoor_yolov9s63, nbs=64, nms=Fa

d:\IT_courses\GOIT\Python\Pandas\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-24 11:39:53,221	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-24 11:39:55,952	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     31104  ultralytics.nn.modules.block.ELAN1           [64, 64, 64, 32]              
  3                  -1  1     73984  ultralytics.nn.modules.block.AConv           [64, 128]                     
  4                  -1  1    258432  ultralytics.nn.modules.block.RepNCSPELAN4    [128, 128, 128, 64, 3]        
  5                  -1  1    221568  ultralytics.nn.modules.block.AConv           [128, 192]                    
  6                  -1  1    579648  ultralytics.nn.modules.block.RepNCSPELAN4    [192, 192, 192, 96, 3]        
  7                  -1  1    442880  ultralytic

train: Scanning D:\IT_courses\GOIT\Python\Pandas\data\train\labels.cache... 892 images, 120 backgrounds, 0 corrupt: 100%|██████████| 1012/1012 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.00.0 ms, read: 456.9116.9 MB/s, size: 217.7 KB)


...

      50/50      5.43G     0.9282     0.5164      1.124         32        640: 100%|██████████| 64/64 [16:02<00:00, 15.04s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:10<00:00,  1.32s/it]

                   all        230       1289      0.509      0.499      0.456      0.276



50 epochs completed in 14.414 hours.
Optimizer stripped from runs\detect\indoor_yolov9s63\weights\last.pt, 15.2MB
Optimizer stripped from runs\detect\indoor_yolov9s63\weights\best.pt, 15.2MB

Validating runs\detect\indoor_yolov9s63\weights\best.pt...
Ultralytics 8.3.167  Python-3.12.6 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
YOLOv9s summary (fused): 197 layers, 7,170,958 parameters, 0 gradients, 26.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:10<00:00,  1.35s/it]


                   all        230       1289       0.38      0.562      0.478      0.304
                  door         63         97      0.355      0.639      0.505      0.317
             open_door         99        765      0.625      0.816      0.808      0.464
          cabinet_door         85        192      0.616      0.714      0.684      0.466
           fridge_door         39         91      0.313      0.455      0.293      0.175
                window         24         49      0.467      0.633      0.613      0.342
                 chair         30         40      0.195      0.425      0.245      0.135
                 table         28         32      0.262      0.594      0.475      0.288
               cabinet          1          1      0.794          1      0.995      0.796
                  sofa         13         13      0.064      0.231     0.0922     0.0372
                pillar          4          9      0.111      0.111     0.0727     0.0248
Speed: 0.6ms preproce

In [16]:
# 3.4 Побудова графіків метрик
csv_path = 'runs/detect/indoor_yolov9s63/results.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()

    # Вибираємо наявні метрики для графіку
    metrics_to_plot = [
        'metrics/mAP50(B)', 
        'metrics/precision(B)', 
        'metrics/recall(B)'
    ]

    # Побудова графіку
    df[metrics_to_plot].plot(figsize=(10, 6), title='Training Metrics')
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("[WARNING] results.csv not found. Skipping metric plots.")

<Figure size 1000x600 with 1 Axes>

In [24]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# 3.5 Оцінка моделі
# Валідація
print("[INFO] Running validation...")
metrics = model.val()

# Шляхи
val_img_dir = './data/val/images'
val_label_dir = './data/val/labels'

image_paths = sorted(glob.glob(os.path.join(val_img_dir, '*.jpg')))
label_paths = sorted(glob.glob(os.path.join(val_label_dir, '*.txt')))

y_true = []
y_pred = []

print("[INFO] Running prediction and collecting labels...")
for img_path, lbl_path in zip(image_paths, label_paths):
    with open(lbl_path, 'r') as f:
        label_line = f.readline()
        if not label_line:
            continue
        true_class = int(label_line.split()[0])
        y_true.append(true_class)

    result = model.predict(img_path, conf=0.25, iou=0.5, save=False, verbose=False)[0]

    if len(result.boxes) > 0:
        pred_class = int(result.boxes.cls[0].item())
    else:
        pred_class = -1  # нічого не знайдено

    y_pred.append(pred_class)

# Видаляємо випадки без передбачення (-1)
y_true_filtered = []
y_pred_filtered = []
for yt, yp in zip(y_true, y_pred):
    if yp != -1:
        y_true_filtered.append(yt)
        y_pred_filtered.append(yp)

# Завантаження класів
with open('indoor.yaml', 'r') as f:
    class_names = yaml.safe_load(f)['names']
num_classes = len(class_names)

# Побудова confusion matrix (з фіксованою кількістю класів)
print("[INFO] Building confusion matrix...")
cm = confusion_matrix(y_true_filtered, y_pred_filtered, labels=list(range(num_classes)))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(include_values=True, xticks_rotation=45, cmap='Blues')
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

# Візуалізація метрик по класах
print("[INFO] Plotting class-wise metrics...")
def pad_list(arr, length, fill=0.0):
    arr = np.array(arr)
    if arr.size < length:
        padded = np.pad(arr, (0, length - arr.size), constant_values=fill)
        return padded.tolist()
    else:
        return arr.tolist()

precision = pad_list(metrics.box.p, num_classes)
recall = pad_list(metrics.box.r, num_classes)
f1 = pad_list(metrics.box.f1, num_classes)
ap = pad_list(metrics.box.ap, num_classes)

x = np.arange(num_classes)

plt.figure(figsize=(12, 6))
plt.plot(x, precision, marker='o', label='Precision')
plt.plot(x, recall, marker='s', label='Recall')
plt.plot(x, f1, marker='^', label='F1-score')
plt.plot(x, ap, marker='x', label='mAP')

plt.xticks(x, class_names, rotation=45)
plt.xlabel("Class")
plt.ylabel("Metric Value")
plt.title("Class-wise Metrics")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

[INFO] Running validation...
Ultralytics 8.3.167  Python-3.12.6 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 828.9515.5 MB/s, size: 198.7 KB)


val: Scanning D:\IT_courses\GOIT\Python\Pandas\data\val\labels.cache... 199 images, 31 backgrounds, 0 corrupt: 100%|██████████| 230/230 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:11<00:00,  1.31it/s]


                   all        230       1289      0.385      0.567       0.48      0.305
                  door         63         97       0.35      0.629      0.504      0.316
             open_door         99        765      0.631      0.818      0.811      0.465
          cabinet_door         85        192       0.61      0.714      0.682      0.464
           fridge_door         39         91      0.311       0.44       0.29      0.174
                window         24         49      0.477      0.653      0.615      0.347
                 chair         30         40      0.206       0.45      0.249      0.132
                 table         28         32      0.281      0.625      0.489      0.294
               cabinet          1          1      0.803          1      0.995      0.796
                  sofa         13         13     0.0641      0.231     0.0926     0.0374
                pillar          4          9      0.112      0.111     0.0729     0.0248
Speed: 0.6ms preproce

<Figure size 640x480 with 2 Axes>

[INFO] Plotting class-wise metrics...


<Figure size 1200x600 with 1 Axes>

In [ ]:
# 3.6 Збереження фінальної моделі
best_weights = getattr(model, 'ckpt_path', 'runs/detect/indoor_yolov9s6/weights/best.pt')
print(f"[INFO] Model saved at: {best_weights}")

[INFO] Model saved at: yolov9s.pt


In [ ]:
# 3.7 Тестування на нових зображеннях
if os.path.exists('data/test/images/'):
    print("[INFO] Running predictions on test images...")
    model.predict('data/test/images/', save=True, conf=0.4)
    print("[INFO] Predictions saved to runs/detect/predict/")
else:
    print("[WARNING] Test image folder not found. Skipping prediction.")

[INFO] Running predictions on test images...

image 1/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1003.png: 480x640 2 open_doors, 128.3ms
image 2/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1014.png: 480x640 (no detections), 77.2ms
image 3/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1015.png: 480x640 (no detections), 75.6ms
image 4/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1020.png: 480x640 (no detections), 71.8ms
image 5/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1021.png: 480x640 (no detections), 76.5ms
image 6/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1023.png: 480x640 1 fridge_door, 1 window, 70.9ms
image 7/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1026.png: 480x640 1 door, 1 open_door, 2 fridge_doors, 2 cabinets, 74.0ms
image 8/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1027.png: 480x640 1 window, 1 cabinet, 70.3ms
image 9/107 d:\IT_courses\GOIT\Python\Pandas\data\test\images\1028.png: 480x640 

In [25]:
# 3.8 Відкриття першого передбаченого зображення (опціонально)
pred_images = sorted(glob.glob('runs/detect/indoor_yolov9s633/*.jpg'))
if pred_images:
    print(f"[INFO] Showing prediction preview: {pred_images[0]}")
    img = Image.open(pred_images[0])
    img.show()
else:
    print("[INFO] No predicted images found to display.")

[INFO] Showing prediction preview: runs/detect/indoor_yolov9s633\1003.jpg


In [13]:
if __name__ == "__main__":
    import multiprocessing
    multiprocessing.freeze_support()  # Для Windows
    main()

[INFO] indoor.yaml already exists — skipping write.


In [ ]:

''' 
1. Аналіз отриманих результатів
1 навчання

Метрика	      Значення
mAP@0.5	        0.48
mAP@0.5:0.95	0.305
Precision (P)	0.385
Recall (R)	    0.567

2 навчання
Метрика	        Значення
mAP@0.5	        0.48
mAP@0.5:0.95	0.304
Precision (P)	0.385
Recall (R)	    0.567


Клас	          P	    R	    mAP@0.5	mAP@0.5:0.95
open_door	    0.631	0.818	0.811	0.465 ✅
cabinet_door	0.610	0.714	0.682	0.464 ✅
window	        0.477	0.653	0.615	0.347 👍
table	        0.281	0.625	0.489	0.294
door	        0.35	0.629	0.504	0.316
fridge_door	    0.311	0.44	0.29	0.174 🔻
chair	        0.206	0.45	0.249	0.132 🔻
sofa	        0.064	0.231	0.093	0.037 ❌
pillar	        0.112	0.111	0.073	0.025 ❌
cabinet	        0.803	1.0	    0.995	0.796 ✅ (але тільки 1 приклад)

mAP@0.5 = 0.48 — цілком прийнятний для невеликого або складного датасету, особливо з 10 класами.

mAP@0.5:0.95 = 0.305 — традиційно нижчий, бо тут метрика жорсткіша (враховує IoU від 0.5 до 0.95).

Precision низький, Recall вищий — модель часто виявляє об'єкти (хороший Recall), але багато помилково класифікує (низький Precision). Це може бути через:

Перекриття класів (наприклад, door vs open_door).
Погану або неоднозначну розмітку.
Модель «перестраховується» і видає багато позитивних детекцій.

Класи open_door, cabinet_door, cabinet — найкраще навчені. Можливо, там більше якісних прикладів.

Проблемні класи: sofa, pillar, chair, fridge_door — усі мають низькі метрикию.
Можливі причини:

Мало прикладів в тренувальному датасеті.
Непослідовна або низькоякісна розмітка.
Класи важко відрізнити візуально (зокрема, chair vs sofa або pillar).
Занадто складна структура цих класів для обраної моделі.

Заключення:
Отримані метрики — відповідають очікуванням для базової моделі на складному датасеті. Основна робота — поліпшення даних та глибше тюнінг архітектури.


2. Оцінка архітектури моделі
Обрана модель — YOLOv9s:

Переваги:
Легка та швидка, з відносно невеликою кількістю параметрів (~7 млн).
Підходить для тренування на невеликих і середніх датасетах.
Добре працює на швидкість і баланс точності.

Можливі недоліки:
Модель "small" може не мати достатньо складної архітектури для розпізнавання тонких відмінностей між схожими класами (door vs open_door, chair vs sofa).
Обмежена здатність до узагальнення на класи з малою кількістю даних.
Використання фіксованого розміру зображення (640x640) може погано відображати дрібні деталі.

3. Шляхи покращення
3.1. Оптимізація датасету
Збільшити кількість зразків проблемних класів (sofa, pillar, chair, fridge_door).
Перевірити та покращити якість розмітки, особливо в проблемних класах.
Виконати аугментації (горизонтальні/вертикальні дзеркала, повороти, зміна яскравості) — допоможе уникнути перенавчання.

3.2. Архітектура та тренування
Спробувати більш потужні версії YOLO (наприклад, YOLOv9m, YOLOv9l) — більша глибина мережі допоможе розпізнавати складні патерни.
Використати додаткові методи регуляризації: DropBlock, label smoothing, MixUp.
Змінити розмір зображень під час тренування — наприклад, збільшити до 768 чи 1024 для кращої деталізації.
Виконати fine-tuning з попередньо натренованою моделлю на великому датасеті (наприклад, COCO).

3.3. Метрики і постобробка
Підібрати оптимальний поріг confidence та NMS (non-max suppression) — це може зменшити помилки класифікації.
Аналізувати помилки за допомогою confusion matrix, виявити, які класи плутаються, і сконцентруватися на їх розділенні.

3.4. Альтернативні підходи
Використати інші архітектури, наприклад, EfficientDet або Cascade R-CNN, для кращої точності.
Використати ансамблі моделей.
Впровадити двокроковий підхід: спершу детекція об'єктів, потім класифікація на підкласи.

4. Загальний висновок:
Тренування моделі - потребує найбільшого ресурсу. 
Графіки та тестові/валідаційні малюнки додані в папку "indoor_yolov9s63".
Навчана модель збережена в файл "yolov9s.pt" та займає дуже великий обсяг.
Вперше використовую файл конфігурації "indoor.yaml" для навчання моделі.
Дуже цікавий досвід. Дякую.
'''

' \n1. Аналіз отриманих результатів\n1 навчання\n\nМетрика\t      Значення\nmAP@0.5\t        0.48\nmAP@0.5:0.95\t0.305\nPrecision (P)\t0.385\nRecall (R)\t    0.567\n\n2 навчання\nМетрика\t        Значення\nmAP@0.5\t        0.48\nmAP@0.5:0.95\t0.304\nPrecision (P)\t0.385\nRecall (R)\t    0.567\n\n\nКлас\t          P\t    R\t    mAP@0.5\tmAP@0.5:0.95\nopen_door\t    0.631\t0.818\t0.811\t0.465 ✅\ncabinet_door\t0.610\t0.714\t0.682\t0.464 ✅\nwindow\t        0.477\t0.653\t0.615\t0.347 👍\ntable\t        0.281\t0.625\t0.489\t0.294\ndoor\t        0.35\t0.629\t0.504\t0.316\nfridge_door\t    0.311\t0.44\t0.29\t0.174 🔻\nchair\t        0.206\t0.45\t0.249\t0.132 🔻\nsofa\t        0.064\t0.231\t0.093\t0.037 ❌\npillar\t        0.112\t0.111\t0.073\t0.025 ❌\ncabinet\t        0.803\t1.0\t    0.995\t0.796 ✅ (але тільки 1 приклад)\n\nmAP@0.5 = 0.48 — цілком прийнятний для невеликого або складного датасету, особливо з 10 класами.\n\nmAP@0.5:0.95 = 0.305 — традиційно нижчий, бо тут метрика жорсткіша (враховує